# Librerías a utilizar

Para la implementación del algoritmo CKY solo serán necesarias dos funciones adicionales, al primera permite seleccionar un elemento de una lista a partir del índice y la segunda para mostrar de manera más clara los resultados. 

In [ ]:
from operator import itemgetter
from pprint import pprint

# Datos de entrada

La variable `file_grammar` que aparece a continuación emula las reglas gramaticales escritas en un archivo de texto. En este caso se han escrito las reglas propias del ejercicio.

In [1]:
file_grammar = """
S -> NP VP
NP -> Det Nominal
NP -> Nominal Nominal
Nominal -> Nominal Noun
Nominal -> Nominal PP
VP -> Verb NP
VP -> Verb PP
PP -> Preposition NP
"""

Se crea el lexicón, de forma común, en lingüística se dice que un elemento del lexicón en la entrada de un diccionario, entonces se define como tal en el código.

In [2]:
lexicon_dictionary = {
    'NP': {'time', 'flies', 'arrow'},
    'Nominal': {'time', 'flies', 'arrow'},
    'VP': {'time', 'flies', 'like'},
    'Verb': {'time', 'flies', 'like'},
    'Noun': {'time', 'flies', 'arrow'},
    'Det': {'an'},
    'Preposition': {'like'}
}

Se expresan las reglas gramaticales y lexicas en forma de tuplas como claves de una probabilidad que se usará para obtener las probabilidades de cada árbol obtenido por el algoritmo. Se eligen tuplas porque son fáciles de manejar, similar a los lenguajes Lisp.

In [3]:
probabilities = {
    ('Det', 'an'): 0.05,
    ('NP', ('Det', 'Nominal')): 0.3,
    ('NP', ('Nominal', 'Nominal')): 0.2,
    ('NP', 'arrow'): 0.002,
    ('NP', 'flies'): 0.002,
    ('NP', 'time'): 0.002,
    ('Nominal', ('Nominal', 'Noun')): 0.1,
    ('Nominal', 'arrow'): 0.002,
    ('Nominal', 'flies'): 0.002,
    ('Nominal', 'time'): 0.002,
    ('Nominal', ('Nominal', 'PP')): 0.2,
    ('Noun', 'arrow'): 0.01,
    ('Noun', 'flies'): 0.01,
    ('Noun', 'time'): 0.01,
    ('PP', ('Preposition', 'NP')): 0.1,
    ('Preposition', 'like'): 0.05,
    ('S', ('NP', 'VP')): 0.8,
    ('VP', ('Verb', 'PP')): 0.2,
    ('VP', 'flies'): 0.008,
    ('VP', 'like'): 0.008,
    ('VP', ('Verb', 'NP')): 0.3,
    ('VP', 'time'): 0.004,
    ('Verb', 'flies'): 0.02,
    ('Verb', 'like'): 0.02,
    ('Verb', 'time'): 0.01
    }

La función `grammar_rules()` recibe como argumento un texto con reglas gramaticales y las convierte en un diccionario que permite buscar a partir de los nodos hijos de un árbol un posible padre que complete la estructura, esta forma es básica para facilitar la búsqueda en el algoritmo CKY.

In [4]:
def grammar_rules(grammar_buffer):
    '''
    This function parse grammar rules writen in a text variable or a file.

    The text file contain grammar rules writen like:

    NP -> NP VP

    This function doesn't parse lexical rules.

    This function return a dictionary with parents indexed by children, like this:

    {('Det', 'Nominal'): ['NP'],
     ('NP', 'VP'): ['S'],
     ('Verb', 'PP'): ['VP']}

        Parameters:
            grammar_buffer (str): Text with grammar rules
        
        Returns:
            find_parent (dict): Dictionary with the rules indexed by children
    '''
    rules = []
    # leer cada línea del texto
    for line in grammar_buffer.strip().split("\n"):
        # separar cada línea en sus dos miembros
        parent, chidren = line.strip().split("->")
        # Quitar espacio en blanco posterior
        parent = parent.strip()
        # Distinguir los hijos
        chidren = chidren.split()
        #crear la regla (parent, (c1, c2)) como en lisp ;)
        rule = (parent, tuple(chidren))
        rules.append(rule)
    # Crear un diccionario con las reglas invertidas, eso permite buscar al nodo padre
    find_parent = dict()
    for parent, (lc, rc) in rules:
        # Si no existe la entrada, se crea
        if (lc, rc) not in find_parent:
            find_parent[lc, rc] = []
        # Se agrega el caso
        find_parent[lc,rc].append(parent)
    return find_parent

#parents = set(val[0] for val in list(find_parent.values()))
#for i in list(lexicon.keys()):
#    parents.add(i)


# Implementación de los algoritmos CKY y PCKY

Ahora se implementa el algoritmo CKY, mismo que recibe como entrada una oración en forma de cadena de texto, un diccionario con reglas gramaticales y un lexicón adecuado a la gramática. Opcionalmente, se puede definir el argumento de entrada `verbose` al valor `True`, de esta manera se mostrará en pantalla cada uno de los pasos que sigue el algoritmo para crear la matriz correspondiente. Esta matriz también tendrá forma de diccionario.

In [5]:
def cky_parser(sentence, grammar, lexicon, verbose=False):
    ''' 
    The function cky_parser implements CKY algorithm.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            verbose (bool): If it is True, show every step in the algorithm
        Returns:
            matrix_cky (dic): Dictionary with all possible cells written by the algorithm
    '''
    words = sentence.lower().split()
    if verbose:
        print(words)
    matrix_cky = dict()
    for i in range(len(words)):
        # Las ordenadas van del 0 al 4
        ordinate = 0
        if verbose:
            print("0: i=",i)
        # Las abscisas van de 1 a 5
        for abscissa in range(i+1, len(words)+1):
            # Se agrega una celda
            matrix_cky[(ordinate, abscissa)] = []
            if verbose:
                print("1: abscissa=",abscissa,"ordinate=",ordinate,"a-o=",abscissa-ordinate)
            # Si se opera sobre la diagonal
            if(abscissa-ordinate==1):
                # Combina la palabra con las categorías del lexicón
                for key in lexicon_dictionary:
                    if verbose:
                        print("2: key=",key,"word=",words[ordinate])
                    # Si la combinación existe en el lexicón se agrega a la matriz
                    if(words[ordinate] in lexicon[key]):
                        matrix_cky[(ordinate,abscissa)].append(
                            (key,0,words[ordinate],words[ordinate]))
                        if verbose:
                            print("2:",matrix_cky)
            # Si hay que operar sobre dos celdas
            elif(abscissa-ordinate>1):
                if verbose:
                    print("3:","(",ordinate,",",abscissa,")")
                # Obtener los valores de las celdas
                for index in range(abscissa-ordinate-1):
                    left = matrix_cky[(ordinate,abscissa-1-index)]
                    down = matrix_cky[(abscissa-1-index,abscissa)]
                    if verbose:
                        print("4: index=",index)
                        print("4: left=",left,"\n  ","down=",down)
                    # Si una de las celdas esta vacía no se hace nada
                    if not left or not down:
                        if verbose:
                            print("5: Nothing to do")
                    else:
                        # Combinar los valores de las celdas
                        for a in left:
                            for b in down:
                                # Obtiene el primer elemento de la tupla
                                # Crea la tupla para buscarla en las reglas
                                if((a[0],b[0]) in grammar):
                                    matrix_cky[(ordinate,abscissa)].append(
                                        (grammar[(a[0],b[0])][0],
                                        abscissa-1-index,a[0],b[0]))
                                    if verbose:
                                        print("6: add=",grammar[(a[0],b[0])][0],abscissa-1-index,a[0],b[0])
                                else:
                                    if verbose:
                                        print("6: Nothing to do")
            if verbose:
                print(matrix_cky)
                print("9: Next Ordinate")
            ordinate+=1
    return matrix_cky

Enseguida se crean dos funciones, la primera, `get_prob()`, se usará para obtener las probabilidades del diccionario de probabilidades, la segunda, `update_prob()`, se usará para actualizar las probabilidades desde los nodos intermedios hasta el nodo padre. 

In [6]:
def get_prob(node, prob_dict):
    '''
    The function get_prob gets a probability from a dictionary.

        Parameters:
            node (tuple): It's a grammar or lexical rule writen in tuple format
            prob_dict (dict): Dictionary with rules as keys an probabilities as values
        Returns:
            prob_dict[node] (float): A probability
    '''
    return prob_dict[node]

def update_prob(left,down,actual, prob_dict):
    '''
        The function get_prob gets a probability from a dictionary.

                Parameters:
                    left (tuple): A tuple with information of a left node
                    down (tuple): A tuple with information of a right node
                    actual (str): Parent node value
                    prob_dict (dict): Dictionary with rules as keys an probabilities as values
                Returns:
                    prob_dict[node] (float): A probability
    '''

    actual_prob = get_prob((actual, (left[0],down[0])), prob_dict)
    prob = left[4] * down[4] * actual_prob
    return prob

Se actualiza `cky_parser()`. Ahora `pcky_parser()` agregará la información de la probabilidad a cada lista agregada a `matrix_pcky`.

In [7]:
def pcky_parser(sentence, grammar, lexicon, probabilities_table,verbose=False):
    ''' 
    The function pcky_parser implements PCKY algorithm.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            probabilities_table (dic): Dictionary with the probability of lexical and gramar rules
            verbose (bool): If it is True, show every step in the algorithm
        Returns:
            matrix_pcky (dic): Dictionary with all possible cells written by the algorithm
    '''
    words = sentence.lower().split()
    if verbose:
        print(words)
    matrix_pcky = dict()
    for i in range(len(words)):
        # Las ordenadas van del 0 al 4
        ordinate = 0
        if verbose:
            print("0: i=",i)
        # Las abscisas van de 1 a 5
        for abscissa in range(i+1, len(words)+1):
            # Se agrega una celda
            matrix_pcky[(ordinate, abscissa)] = []
            if verbose:
                print("1: abscissa=",abscissa,"ordinate=",ordinate,"a-o=",abscissa-ordinate)
            # Si se opera sobre la diagonal
            if(abscissa-ordinate==1):
                # Combina la palabra con las categorías del lexicón
                for key in lexicon_dictionary:
                    if verbose:
                        print("2: key=",key,"word=",words[ordinate])
                    # Si la combinación existe en el lexicón se agrega a la matriz
                    if(words[ordinate] in lexicon[key]):
                        matrix_pcky[(ordinate,abscissa)].append(
                            (key,0,words[ordinate],words[ordinate],
                            get_prob((key,words[ordinate]),probabilities_table)))
                        if verbose:
                            print("2:",matrix_pcky)
            # Si hay que operar sobre dos celdas
            elif(abscissa-ordinate>1):
                if verbose:
                    print("3:","(",ordinate,",",abscissa,")")
                # Obtener los valores de las celdas
                for index in range(abscissa-ordinate-1):
                    left = matrix_pcky[(ordinate,abscissa-1-index)]
                    down = matrix_pcky[(abscissa-1-index,abscissa)]
                    if verbose:
                        print("4: index=",index)
                        print("4: left=",left,"\n  ","down=",down)
                    # Si una de las celdas esta vacía no se hace nada
                    if not left or not down:
                        if verbose:
                            print("5: Nothing to do")
                    else:
                        # Combinar los valores de las celdas
                        for a in left:
                            for b in down:
                                # Obtiene el primer elemento de la tupla
                                # Crea la tupla para buscarla en las reglas
                                if((a[0],b[0]) in grammar):
                                    matrix_pcky[(ordinate,abscissa)].append(
                                        (grammar[(a[0],b[0])][0],
                                        abscissa-1-index,a[0],b[0],
                                        update_prob(a,b,grammar[(a[0],b[0])][0],probabilities_table))
                                        )
                                    if verbose:
                                        print("6: add=",grammar[(a[0],b[0])][0],abscissa-1-index,a[0],b[0])
                                else:
                                    if verbose:
                                        print("6: Nothing to do")
            if verbose:
                print(matrix_pcky)
                print("9: Next Ordinate")
            ordinate+=1
    return matrix_pcky

# Obtención de resultados

La función `tree()` recorre una matriz generada por el algoritmo CKY en busca de árboles a partir del valor de un nodo en particular, tomando en cuenta una ruta a través de un índice creado por el algoritmo. 

In [8]:
def tree (matrix,cell,parent="S",index=0,pcky=False):
    '''
    Create a tree from a matrix created by the CKY algorithm.
    See def cky_parser(sentence, grammar, lexicon, verbose=False) function.

        Parameters:
            matrix (dic): Dictionary with CKY algorithm information
            cell (tuple): Node in the matrix to start the analysis
            parent (str): Axiom in the grammar or parent node value for the tree
            index (int): If there is more than one value in cell, this variable let choose which one analyze
            pcky (bool): Enable if matrix contains PCKY algorithm information
        Returns:
            list (list): A list with trees in list shape
    '''
    list = []
    # Revisa que la lista no este vacía
    if len(matrix[cell]) == 0:
        return list
    # Revisa que el índice sea 0, significa que es una hoja
    if matrix[cell][0][1] == 0:
        # Encuentra la hoja que corresponde
        if matrix[cell][index][0] == parent:
            if pcky:
                return [(matrix[cell][index][0],matrix[cell][index][4]), matrix[cell][index][2]]
            else:
                return [matrix[cell][index][0], matrix[cell][index][2]]
        else:
            # Busca en otra hoja
            if index+1 < len(matrix[cell]):
                if pcky:
                    return tree(matrix,cell,parent,index+1,pcky=True)
                else:
                    return tree(matrix,cell,parent,index+1)
            else:
                return list
    # Si no es una hoja, revisa si es la opción correcta
    elif (matrix[cell][index][0]) == parent:
        # Agrega el símbolo
        if pcky:
            list.append((matrix[cell][index][0],matrix[cell][index][4]))
        else:
            list.append(matrix[cell][index][0])
        # Buscar hijos
        child = []
        # Hijo derecho
        if pcky:
            child.append(
                tree(matrix,(cell[0],matrix[cell][index][1]),matrix[cell][index][2],pcky=True))
        else:
            child.append(
                tree(matrix,(cell[0],matrix[cell][index][1]),matrix[cell][index][2]))
        # Hijo izquierdo
        if pcky:
            child.append(
                tree(matrix,(matrix[cell][index][1],cell[1]),matrix[cell][index][3],pcky=True))
        else:
            child.append(
                tree(matrix,(matrix[cell][index][1],cell[1]),matrix[cell][index][3]))
        #Agrega child a la lista
        list.append(child)
        return list
    else:
        if index+1 < len(matrix[cell]):
            if pcky:
                return tree(matrix,cell,parent,index+1,pcky=True)
            else:
                return tree(matrix,cell,parent,index+1)
        else:
            return list

La función `find_solutions()` servirá como _frontend_ para todas las funciones anteriores, brindará la respuesta propuesta por el algoritmo.

In [9]:
def find_solutions(sentence,grammar,lexicon,axiom,probabilities_table={},verbose=False):
    '''
    Find all possible solutions of a CKY analysis in a sentence.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            axiom (str): Axiom in the grammar
            probabilities_table (dic): Dictionary with the probability of lexical and gramar rules
            verbose (bool): If it is True, show every step in the algorithm

        Returns:
            solutions (list): List with all possible solutinos suggested by the algorithm
    '''
    N = len(sentence.split())
    solutions = []
    if probabilities_table:
        matrix_pcky = pcky_parser(sentence,grammar,lexicon,probabilities_table,verbose)
        candidates = []
        # Busca en (0,N) aquellas listas que contengan el axioma
        for i in enumerate(matrix_pcky[(0,N)]):
            if i[1][0] == axiom:
                candidates.append([i[1][0],i[1][1],i[1][4]])
        # Selecciona aquel con mayor probabilidad
        candidates = max(candidates,key=itemgetter(2))
        solutions = tree(matrix_pcky,(0,N),index=candidates[1],pcky=True)
    else:
        matrix_cky = cky_parser(sentence,grammar,lexicon,verbose)
        for i in enumerate(matrix_cky[(0,N)]):
            if i[1][0] == axiom:
                solutions.append(tree(matrix_cky,(0,N),index=i[0]))
    return solutions

In [10]:
options = find_solutions("Time flies like an arrow",
                        grammar_rules(file_grammar),
                        lexicon_dictionary,
                        "S")
pprint(options)

[['S',
  [['NP', [['Nominal', 'time'], ['Nominal', 'flies']]],
   ['VP', [['Verb', 'like'], ['NP', [['Det', 'an'], ['Nominal', 'arrow']]]]]]],
 ['S',
  [['NP', 'time'],
   ['VP',
    [['Verb', 'flies'],
     ['PP',
      [['Preposition', 'like'],
       ['NP', [['Det', 'an'], ['Nominal', 'arrow']]]]]]]]]]


In [11]:
<options = find_solutions("Time flies like an arrow",
                        grammar_rules(file_grammar),
                        lexicon_dictionary,
                        "S",probabilities)
pprint(options)

[('S', 9.600000000000002e-13),
 [[('NP', 0.002), 'time'],
  [('VP', 6.000000000000001e-10),
   [[('Verb', 0.02), 'flies'],
    [('PP', 1.5000000000000002e-07),
     [[('Preposition', 0.05), 'like'],
      [('NP', 3e-05),
       [[('Det', 0.05), 'an'], [('Nominal', 0.002), 'arrow']]]]]]]]]


# Test code

In [12]:
cky = pcky_parser("Time flies like an arrow", grammar_rules(file_grammar), lexicon_dictionary,probabilities)

In [13]:
cky = cky_parser("Time flies like an arrow", grammar_rules(file_grammar), lexicon_dictionary)

In [14]:
import numpy as np
import pandas as pd

# pares de puntos de la matriz
dic_to_df = dict()
for i in range(0,5):
    list = []
    for j in range(0,5-i):
        #print((i,j+i+1),":",cky[(i,j+i+1)])
        list.append(cky[(i,j+i+1)])
        dic_to_df[str(i)] = list

N=5
df = pd.DataFrame(columns = [i for i in range(0,N)], 
                   index = [i for i in range(1,N+1)])

df[0] = dic_to_df['0']
print(df)

#for i in range(0,5):
#    df[i] = dic_to_df[str(i)]

                                                   0    1    2    3    4
1  [(NP, 0, time, time), (Nominal, 0, time, time)...  NaN  NaN  NaN  NaN
2  [(S, 1, NP, VP), (NP, 1, Nominal, Nominal), (N...  NaN  NaN  NaN  NaN
3                                   [(S, 2, NP, VP)]  NaN  NaN  NaN  NaN
4                                                 []  NaN  NaN  NaN  NaN
5  [(S, 2, NP, VP), (Nominal, 2, Nominal, PP), (S...  NaN  NaN  NaN  NaN


# Cuestionario

1. ¿Es correcto el análisis sintáctico que se ha obtenido? Justifica la respuesta.

    HOLA

2. ¿Cuáles son las limitaciones de aplicar el algoritmo CKY probabilístico para realizar el análisis sintáctico? Justifica la respuesta.

    HOLA

3. ¿Qué posibles mejoras que se podrían aplicar para mejorar el rendimiento del análisis sintáctico? Justifica la respuesta.

    HOLA

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=16bbac0e-c0cf-475a-9977-7067c93e2eaf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>